In [22]:
import pandas as pd

PATH_DATA = 'data/'
FILE = 'votacao_candidato_munzona_2022_BRASIL.csv'

In [23]:
colunas_relevantes = ["NR_TURNO", "DS_ELEICAO", "TP_ABRANGENCIA", "SG_UF", "NM_MUNICIPIO", "NR_ZONA", "DS_CARGO", "NR_CANDIDATO", "NM_CANDIDATO", "NM_URNA_CANDIDATO", "DS_SITUACAO_CANDIDATURA",
                      "NR_PARTIDO", "SG_PARTIDO", "NM_PARTIDO", "NM_COLIGACAO", "DS_COMPOSICAO_COLIGACAO", "ST_VOTO_EM_TRANSITO", "QT_VOTOS_NOMINAIS", "NM_TIPO_DESTINACAO_VOTOS", "QT_VOTOS_NOMINAIS_VALIDOS", "DS_SIT_TOT_TURNO"]
colunas_relevantes_2 = ["NR_TURNO","SG_UF","NM_MUNICIPIO","NR_CANDIDATO","NM_URNA_CANDIDATO","SG_PARTIDO","ST_VOTO_EM_TRANSITO","QT_VOTOS_NOMINAIS_VALIDOS","DS_SIT_TOT_TURNO","TP_AGREMIACAO","DS_CARGO"]
colunas_relevantes_2.sort()

In [31]:
import time
def create_monitor_file():
    header = "type,function,seconds"
    try:
        with open("monitor.txt", "x") as file:
            file.write(header)
    except FileExistsError:
        pass
def execution_time_logger(name,tipo):
    def decorator(func):
        def wrapper(*args, **kwargs):
            start_time = time.time()
            result = func(*args, **kwargs)
            end_time = time.time()
            execution_time = round(end_time - start_time,2)

            create_monitor_file()
            with open("monitor.txt", "a") as file:
                file.write(f'\n{tipo},{name},{execution_time}')
            return result
        return wrapper
    return decorator


In [39]:
@execution_time_logger(name="leitura",tipo='pandas')
def read():
    df = pd.read_csv(PATH_DATA +FILE,delimiter=";",decimal=",",encoding="latin-1",usecols=colunas_relevantes_2)
    return df

@execution_time_logger(name="transformacao",tipo='pandas')
def transforma(df):
    def transforma_uf(uf):
        if uf == 'EXTERIOR':
            return uf
        else:
            return  f'BRASIL - {uf}'

    df1 = df.copy()

    #Ajuste de Variáveis
    df1.loc[df1['SG_UF'] =='ZZ','SG_UF'] = 'EXTERIOR'
    df1['SG_UF_DETALHAMENTO'] = df1['SG_UF'].apply(lambda x:transforma_uf(x))
    df1.loc[df1['SG_UF'] == 'EXTERIOR','NM_MUNICIPIO'] = 'EXTERIOR'

    # Agrupamento
    colunas_agrupar = list(df1.columns.values).copy()
    colunas_agrupar.remove('QT_VOTOS_NOMINAIS_VALIDOS')
    df1 = df1.groupby(colunas_agrupar).sum().reset_index()

    #Ajuste de Variáveis
    df1['ST_VOTO_EM_TRANSITO'] = df1['ST_VOTO_EM_TRANSITO'].replace({'N': 'NÃO','S':'SIM'})
    df1['NR_TURNO'] = df1['NR_TURNO'].replace({1: 'PRIMEIRO',2:'SEGUNDO'})
    df1['DS_SIT_TOT_TURNO'] = df1['DS_SIT_TOT_TURNO'].replace({'ELEITO POR QP': 'ELEITO POR QUOCIENTE PARTIDÁRIO'})

    #Maiúscula
    df1['DS_CARGO'] = df1['DS_CARGO'].str.upper()
    df1['TP_AGREMIACAO'] = df1['TP_AGREMIACAO'].str.upper()
    return df1

@execution_time_logger(name="parquet",tipo='pandas')
def save(df1):
    SAVE_FOLDER = 'data/servico/'
    df1.to_parquet(SAVE_FOLDER+'eleicoes_servico.parquet')

In [42]:
for i in range(10):
    df = read()
    df1 = transforma(df)
    del df
    save(df1)
    del df1
    print(i)

/opt/conda/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


0


/opt/conda/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


1


/opt/conda/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


...

7


/opt/conda/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


8


/opt/conda/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


9


In [35]:

df1.to_parquet(SAVE_FOLDER+'eleicoes_servico.parquet.gzip',compression='gzip')
df1.to_csv(SAVE_FOLDER+'eleicoes_servico.csv')

- Qual a quantidade total de votos por partido (top 10 gráfico de barras)? SG_PARTIDO,QT_VOTOS_NOMINAIS_VALIDOS
- Quais são os estados (UF) com o maior e menor número de votos válidos (mapa de calor)? SG_UF,QT_VOTOS_NOMINAIS_VALIDOS
- Quais os dez municípios com a maior quantidade de votos e os dez com a menor quantidade (livre)? NM_MUNICIPIO,QT_VOTOS_NOMINAIS_VALIDOS
- Qual a quantidade de votos nominais por partido (PARTIDO) e como isso varia entre os diferentes estados (UF) (top três por estado – livre)? SG_UF,QT_VOTOS_NOMINAIS_VALIDOS,SG_PARTIDO
- Quais os três municípios (MUNICIPIO) com a maior taxa de votos em trânsito (VOTO_EM_TRANSITO) em relação ao total de votos válidos do município (livre)? NM_MUNICIPIO,QT_VOTOS_NOMINAIS_VALIDOS,ST_VOTO_EM_TRANSITO
- Qual a quantidade de votos por turno (pizza)? NR_TURNO,QT_VOTOS_NOMINAIS_VALIDOS
- Qual os 5 números de candidatos com maior frequência (gráfico de barras horizontais)? NM_URNA_CANDIDATO
- Quais os dez partidos com maior número de votos por estado (gráfico de barras)?  SG_PARTIDO,QT_VOTOS_NOMINAIS_VALIDOS,SG_UF
- Qual foi o candidato com partido e estado com maior número de votos nominais em uma única eleição por cargo (livre)?  SG_PARTIDO,NM_URNA_CANDIDATO,QT_VOTOS_NOMINAIS_VALIDOS,SG_UF,NR_TURNO
- Qual a quantidade de candidatos eleitos que participaram de coligações e a quantidade de candidatos eleitos sem coligação (pizza)? TP_AGREMIACAO,DS_SIT_TOT_TURNO